# Deep learning NER

In this notebook, we will discover two deep learning techniques for Named Entity Recognition (or NER). 

- LSTM (Long Short Term Memory)
- Transformers

In [5]:
from app.travel_resolver.libs.nlp import data_processing as dp

sentences, labels, vocab, unique_labels = dp.from_bio_file_to_examples('./data/bio/fr.bio/fr.sentences.bio')

In [6]:
processed_sentences = []
processed_labels = []

for sentence, label in zip(sentences, labels):
    sentence, label = dp.process_sentence(sentence, stemming=True, return_tokens=True, labels_to_adapt=label)
    processed_sentences.append(sentence)
    processed_labels.append(label)


In [ ]:
def encode_sentence(sentence: str, vocab: list[str]):
    """
    Encode a sentence into a list of integers

    Parameters:
    sentence (str): The sentence to encode
    vocab (list): The vocabulary

    Returns:
    list: The list of integers
    """
    return [
        vocab.index(word) if word in vocab else vocab.index("<UNK>")
        for word in sentence.split(" ")
    ]